# setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mahotas

     |████████████████████████████████| 5.7 MB 4.7 MB/s 


In [3]:
!pip install catboost

     |████████████████████████████████| 76.4 MB 48 kB/s 


In [4]:
!pip install radiant_mlhub

     |████████████████████████████████| 201 kB 6.0 MB/s 
     |████████████████████████████████| 62 kB 769 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
!pip install rasterio

     |████████████████████████████████| 19.3 MB 5.2 MB/s 


In [6]:

# Required libraries
import os
import tarfile
import json
import pandas as pd
from pathlib import Path
from radiant_mlhub.client import _download as download_file

import datetime
import rasterio
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import ndimage 
import numpy.ma as ma
import mahotas

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
!pip install opencv-python numpy matplotlib

In [9]:
import cv2

# load data

In [10]:
DOWNLOAD_S1 = True # If you set this to true then the Sentinel-1 data will be downloaded

# Select which imagery bands you'd like to download here
DOWNLOAD_BANDS = {
    'B01': True,
    'B02': True,
    'B03': True,
    'B04': True,
    'B05': True,
    'B06': True,
    'B07': True,
    'B08': True,
    'B8A': True,
    'B09': True,
    
    'B11': True,
    'B12': True,
    'CLM': True
}

In [82]:
os.environ['MLHUB_API_KEY'] = 'a6bb13078fcfa091ebd00341a91752e560a434b58cab5daf5ac1ccdacc836dac'
FOLDER_BASE = 'ref_south_africa_crops_competition_v1'

def download_archive(archive_name):
    if os.path.exists(archive_name.replace('.tar.gz', '')):
        return
    
    print(f'Downloading {archive_name} ...')
    download_url = f'https://radiant-mlhub.s3.us-west-2.amazonaws.com/archives/{archive_name}'
    download_file(download_url, '.')
    print(f'Extracting {archive_name} ...')
    with tarfile.open(archive_name) as tfile:
        tfile.extractall()
    os.remove(archive_name)

for split in ['train']:
    # Download the labels
    labels_archive = f'{FOLDER_BASE}_{split}_labels.tar.gz'
    download_archive(labels_archive)
    
    # Download Sentinel-1 data
    if DOWNLOAD_S1:
        s1_archive = f'{FOLDER_BASE}_{split}_source_s1.tar.gz'
        download_archive(s1_archive)
        

    for band, download in DOWNLOAD_BANDS.items():
        if not download:
            continue
        s2_archive = f'{FOLDER_BASE}_{split}_source_s2_{band}.tar.gz'
        download_archive(s2_archive)
        
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
        
def load_df(collection_id):
    split = collection_id.split('_')[-2]
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            source_item_id = link['href'].split('/')[-2]
            
            if source_item_id.find('_s1_') > 0 and not DOWNLOAD_S1:
                continue
            elif source_item_id.find('_s1_') > 0:
                for band in ['VV', 'VH']:
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s1/{source_item_id}/{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's1',
                        band,
                        asset_path
                    ])
                
            if source_item_id.find('_s2_') > 0:
                for band, download in DOWNLOAD_BANDS.items():
                    if not download:
                        continue
                    
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s2_{band}/{source_item_id}_{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's2',
                        band,
                        asset_path
                    ])
            
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

train_df = load_df(f'{FOLDER_BASE}_train_labels')
#test_df = load_df(f'{FOLDER_BASE}_test_labels')

  0%|          | 0/6755.8 [00:00<?, ?M/s]

Extracting ref_south_africa_crops_competition_v1_train_source_s2_B08.tar.gz ...


# train

In [12]:
def time(data):
  data['datetime']=pd.to_datetime(data['datetime'])
  data['month']=data['datetime'].dt.month
  data['week']=data['datetime'].dt.isocalendar().week

In [13]:
time(train_df)
train_df['day']=train_df['datetime'].dt.day 
train_df['fortnight']=np.where(train_df.day<15,1,2)

In [ ]:
label_df=pd.read_csv(train_df[(train_df.asset=='field_info_train')&(train_df.tile_id=='0001')]['file_path'].values[0])
label_df.rename(columns={'Field ID':'field_id','Crop':'label'},inplace=True)

label_map={'Lucerne/Medics':1,'Planted pastures (perennial)':2,'Fallow':3,'Wine grapes':4,'Weeds':5,'Small grain grazing':6,'Wheat':7,'Canola':8,'Rooibos':9}
label_df['label']=label_df['label'].map(label_map)

# Sentinel 1 exploration

In [15]:
sentinel1_train=train_df[train_df.satellite_platform=='s1']

In [16]:
sentinel1_fortnight=sentinel1_train.groupby(['tile_id','month','fortnight','asset']).first().reset_index()

In [17]:
info=train_df[(train_df.asset=='field_ids')|(train_df.asset=='labels')]



In [45]:
def sentinel1(data,month0=4):
  field_ndvi=[]
  for date in data.month.unique():
    month_i=[]
    for tile in data.tile_id.unique():
     
      tile_df=data[(data.tile_id==tile) & (data.month==date)]
      

      VV1=rasterio.open(tile_df[(tile_df['asset']=='VV')&(tile_df['fortnight']==1)]['file_path'].values[0]).read(1)
      VH1=rasterio.open(tile_df[(tile_df['asset']=='VH')&(tile_df['fortnight']==1)]['file_path'].values[0]).read(1)

      VV2=rasterio.open(tile_df[(tile_df['asset']=='VV')&(tile_df['fortnight']==2)]['file_path'].values[0]).read(1)
      VH2=rasterio.open(tile_df[(tile_df['asset']=='VH')&(tile_df['fortnight']==2)]['file_path'].values[0]).read(1)
    
      tile_fields=rasterio.open(info[(info.tile_id==tile)&(info.asset=='field_ids')]['file_path'].values[0]).read(1)
      ids=np.unique(tile_fields.flatten())

      for field in ids:
        if (field==0):
          continue
       
        
        mask_VV1=ma.masked_array(VV1, np.where(tile_fields==field,0,1).astype(int))
        mask_VH1=ma.masked_array(VH1, np.where(tile_fields==field,0,1).astype(int))
        
        mask_VV2=ma.masked_array(VV2, np.where(tile_fields==field,0,1).astype(int))
        mask_VH2=ma.masked_array(VH2, np.where(tile_fields==field,0,1).astype(int))
        
        if (date==month0):
          
          field_ndvi.append((tile,field,np.mean(mask_VV1),np.mean(mask_VH1),np.mean(mask_VV2),np.mean(mask_VH2)))

        else:
          month_i.append([np.mean(mask_VV1),np.mean(mask_VH1),np.mean(mask_VV2),np.mean(mask_VH2)])



    if (date==month0):
      train=pd.DataFrame(field_ndvi,columns=['tile_id','field_id',f'VV1_median_{int(date)}',f'VH1_median_{int(date)}',f'VV2_median_{int(date)}',f'VH2_median_{int(date)}'])
    else: 
      train[[f'VV1_median_{int(date)}',f'VH1_median_{int(date)}',f'VV2_median_{int(date)}',f'VH2_median_{int(date)}']]=month_i

  

  return train

In [ ]:
#executing this cell will take time, the output of this cell is cloudy(dataframe) which can be directly obtained
#by executing the next cell
sentinel1_ft=sentinel1(sentinel1_train)

In [47]:
sentinel1_ft=pd.read_parquet('/content/drive/MyDrive/spot_the_crop/sentinel1_train.parquet')

In [49]:
sentinel1_ft.drop('tile_id',axis=1,inplace=True)

# cloudless image selection

In [14]:
#in each month look for the date in which CLM is all 0s

def cloudless(data):
  for tile in data.tile_id.unique():
    tile_df=data[(data.tile_id==tile)&(data.satellite_platform=='s2')]
    for date in tile_df.datetime.unique():
      src_mask=rasterio.open(tile_df[(tile_df.datetime==date)&(tile_df['asset']=='CLM')]['file_path'].values[0])
      if (np.max(src_mask.read(1))==0):
        data.loc[(data.tile_id==tile) & (data.datetime==date),'cloudless']=0
      if (np.max(src_mask.read(1))==255):
        data.loc[(data.tile_id==tile) & (data.datetime==date),'cloudless']=np.sum(src_mask.read(1))/255

      if (np.max(src_mask.read(1))==1):
        data.loc[(data.tile_id==tile) & (data.datetime==date),'cloudless']=np.sum(src_mask.read(1))+0.02

In [15]:
#executing this cell will take time, the output of this cell is cloudy(dataframe) which can be directly obtained
#by executing the next cell
cloudy=train_df.copy()
cloudless(cloudy)

In [50]:
cloudy=pd.read_csv('/content/drive/MyDrive/spot_the_crop/train_cloudless_fortnight_all.csv')

In [64]:
cloudy.drop('Unnamed: 0',axis=1,inplace=True)

**fortnight cloudless**

In [65]:
#in each month we take the best 2 observations one from each fortnight
cloudy.set_index(['tile_id','month','fortnight'],inplace=True)
cloudy['best']=cloudy.groupby(['tile_id','month','fortnight']).cloudless.min()
cloudy.reset_index(inplace=True)

cloud_free_train=cloudy[(cloudy.best==cloudy.cloudless)].copy()


cloud_free_train=cloud_free_train.groupby(['tile_id','month','fortnight','asset']).first().reset_index()

In [66]:
info=train_df[(train_df.asset=='field_ids')|(train_df.asset=='labels')]
info['tile_id']=info['tile_id'].astype(int).values

**overall cloudless**

In [67]:
cloudy_all=cloudy.copy()
cloudy_all.set_index(['tile_id','month'],inplace=True)
cloudy_all['best']=cloudy_all.groupby(['tile_id','month']).cloudless.min()
cloudy_all.reset_index(inplace=True)

cloud_free_train1=cloudy_all[(cloudy_all.best==cloudy_all.cloudless)].copy()

cloud_free_train1=cloud_free_train1.groupby(['tile_id','month','asset']).first().reset_index()



# Feature engineering

In [54]:
def veg_indices(data,info,month0=4):
  field_ndvi=[]
  for date in data.month.unique():
    print(f'processing for month {date} has began')
    for f in [1,2]:
      month_i=[]
      for tile in data.tile_id.unique():
        
        tile_df=data[(data.tile_id==tile) & (data.month==date) & (data.fortnight==f)]

        src_aerosol=rasterio.open(tile_df[tile_df['asset']=='B01']['file_path'].values[0])
        src_blue=rasterio.open(tile_df[tile_df['asset']=='B02']['file_path'].values[0])
        src_green=rasterio.open(tile_df[tile_df['asset']=='B03']['file_path'].values[0])
        src_red=rasterio.open(tile_df[tile_df['asset']=='B04']['file_path'].values[0])
        src_edge=rasterio.open(tile_df[tile_df['asset']=='B05']['file_path'].values[0])
        src_edge2=rasterio.open(tile_df[tile_df['asset']=='B06']['file_path'].values[0])
        src_edge3=rasterio.open(tile_df[tile_df['asset']=='B07']['file_path'].values[0])       
        src_nir=rasterio.open(tile_df[tile_df['asset']=='B08']['file_path'].values[0])
        src_8a=rasterio.open(tile_df[tile_df['asset']=='B8A']['file_path'].values[0])
        src_vapor=rasterio.open(tile_df[tile_df['asset']=='B09']['file_path'].values[0])
        src_swir1=rasterio.open(tile_df[tile_df['asset']=='B11']['file_path'].values[0])
        src_swir2=rasterio.open(tile_df[tile_df['asset']=='B12']['file_path'].values[0])
        


        ndvi=np.where(src_nir.read(1)+src_red.read(1)==0,0,(src_nir.read(1)-src_red.read(1))/(src_nir.read(1)+src_red.read(1)))
        lswi=np.where(src_nir.read(1)+src_swir1.read(1)==0,0,(src_nir.read(1)-src_swir1.read(1))/(src_nir.read(1)+src_swir1.read(1)))
        ndsvi=np.where(src_red.read(1)+src_swir1.read(1)==0,0,(src_swir1.read(1)-src_red.read(1))/(src_red.read(1)+src_swir1.read(1)))
        ndti=np.where(src_swir2.read(1)+src_swir1.read(1)==0,0,(src_swir1.read(1)-src_swir2.read(1))/(src_swir2.read(1)+src_swir1.read(1)))
        rvi=src_aerosol.read(1)/(src_blue.read(1)+1e-8)
        gli=np.where(2*src_green.read(1)+src_red.read(1)+src_aerosol.read(1)==0,0
                    ,(2*src_green.read(1)-src_red.read(1)-src_aerosol.read(1))/(2*src_green.read(1)+src_red.read(1)+src_aerosol.read(1)))
        
        bsi=100*np.sqrt((src_swir1.read(1)-src_green.read(1))/(src_swir1.read(1)+src_green.read(1)))
        
        MCARI1=np.where(src_red.read(1)==0,0,((src_edge.read(1)-src_red.read(1))-0.2*(src_edge.read(1)-src_green.read(1))*(src_edge.read(1)/src_red.read(1))))
        
        savi=(src_nir.read(1) - src_red.read(1)) / (src_nir.read(1) + src_red.read(1) + 0.725) * (1.0 + 0.725)

        grndvi=(src_green.read(1)-src_red.read(1))/(src_green.read(1)+src_red.read(1))
        grndvi=np.where((src_green.read(1)+src_red.read(1))==0,0,grndvi)
        ndsi=np.where((src_green.read(1)+src_swir1.read(1))==0,0,(src_green.read(1)-src_swir1.read(1))/(src_green.read(1)+src_swir1.read(1)))
        gb=np.where((src_green.read(1)+src_blue.read(1))==0,0,(src_green.read(1)-src_blue.read(1))/(src_green.read(1)+src_blue.read(1)))
        psri=np.where(src_nir.read(1)==0,0,(src_red.read(1)-src_green.read(1))/src_nir.read(1))
        npci=np.where((src_red.read(1)+src_blue.read(1))==0,0,(src_red.read(1)-src_blue.read(1))/(src_red.read(1)+src_blue.read(1)))
        
        ccci=((src_nir.read(1)-src_edge.read(1))/(src_nir.read(1)+src_edge.read(1)))/((src_nir.read(1)-src_red.read(1))/(src_nir.read(1)+src_red.read(1)))
        gcvi=np.where(src_green.read(1)==0,0,(src_nir.read(1)/src_green.read(1))-1)

        clred_edge=src_edge.read(1)/src_edge3.read(1)   
        slavi=src_nir.read(1)/(src_red.read(1)+src_swir2.read(1))
        datt1=(src_nir.read(1)-src_edge.read(1))/(src_nir.read(1)-src_red.read(1))
        arvi=(src_8a.read(1)-src_red.read(1)-0.069*(src_red.read(1)-src_blue.read(1)))/(src_8a.read(1)+src_red.read(1)-0.069*(src_red.read(1)-src_blue.read(1))) 

        
  
        

        tile_fields=rasterio.open(info[(info.tile_id==tile)&(info.asset=='field_ids')]['file_path'].values[0]).read(1)
        ids=np.unique(tile_fields.flatten())

        for field in ids:
          if (field==0):
            continue
          mask_ndvi=ma.masked_array(ndvi, np.where(tile_fields==field,0,1).astype(int))
          mask_lswi=ma.masked_array(lswi, np.where(tile_fields==field,0,1).astype(int))
          mask_ndsvi=ma.masked_array(ndsvi, np.where(tile_fields==field,0,1).astype(int))
          mask_ndti=ma.masked_array(ndti, np.where(tile_fields==field,0,1).astype(int))
          mask_vapor=ma.masked_array(src_vapor.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_gli=ma.masked_array(gli, np.where(tile_fields==field,0,1).astype(int))
          mask_rvi=ma.masked_array(rvi, np.where(tile_fields==field,0,1).astype(int))
          mask_bsi=ma.masked_array(bsi, np.where(tile_fields==field,0,1).astype(int))
          mask_nir=ma.masked_array(src_nir.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_mcari=ma.masked_array(MCARI1, np.where(tile_fields==field,0,1).astype(int))

          mask_savi=ma.masked_array(savi, np.where(tile_fields==field,0,1).astype(int))

          mask_grndvi=ma.masked_array(grndvi, np.where(tile_fields==field,0,1).astype(int))
          mask_ndsi=ma.masked_array(ndsi, np.where(tile_fields==field,0,1).astype(int))
          mask_gb=ma.masked_array(gb, np.where(tile_fields==field,0,1).astype(int))
          mask_psri=ma.masked_array(psri, np.where(tile_fields==field,0,1).astype(int))
          mask_npci=ma.masked_array(npci, np.where(tile_fields==field,0,1).astype(int))

          mask_ccci=ma.masked_array(ccci, np.where(tile_fields==field,0,1).astype(int))
          mask_gcvi=ma.masked_array(gcvi, np.where(tile_fields==field,0,1).astype(int))
          mask_red=ma.masked_array(src_red.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_edge=ma.masked_array(src_edge.read(1), np.where(tile_fields==field,0,1).astype(int))

          mask_green=ma.masked_array(src_green.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_blue=ma.masked_array(src_blue.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_edge2=ma.masked_array(src_edge2.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_edge3=ma.masked_array(src_edge3.read(1), np.where(tile_fields==field,0,1).astype(int))
          mask_8a=ma.masked_array(src_8a.read(1), np.where(tile_fields==field,0,1).astype(int))

          mask_clred_edge=ma.masked_array(clred_edge, np.where(tile_fields==field,0,1).astype(int))
          mask_slavi=ma.masked_array(slavi, np.where(tile_fields==field,0,1).astype(int))
          mask_datt1=ma.masked_array(datt1, np.where(tile_fields==field,0,1).astype(int))
          mask_arvi=ma.masked_array(arvi, np.where(tile_fields==field,0,1).astype(int))

          if ((date==month0)& (f==1)):
            
            field_ndvi.append((tile,field,np.mean(mask_vapor),np.mean(mask_ndvi),np.mean(mask_lswi),
                               np.mean(mask_ndsvi),np.mean(mask_ndti),np.mean(mask_rvi),np.mean(mask_gli),np.std(mask_ndvi),
                               np.mean(mask_bsi),np.mean(mask_nir),np.mean(mask_mcari),np.mean(mask_savi),
                               np.mean(mask_grndvi),np.mean(mask_ndsi),np.mean(mask_gb),np.mean(mask_psri),
                               np.mean(mask_npci),np.mean(mask_ccci),np.mean(mask_gcvi),np.mean(mask_red),
                               np.mean(mask_edge), np.mean(mask_green),np.mean(mask_blue),np.mean(mask_edge2),
                               np.mean(mask_edge3),np.mean(mask_8a),np.std(mask_nir),
                               np.mean(mask_clred_edge),np.mean(mask_slavi),np.mean(mask_datt1),np.mean(mask_arvi)))

          else:
            month_i.append([np.mean(mask_vapor),np.mean(mask_ndvi),np.mean(mask_lswi),
                               np.mean(mask_ndsvi),np.mean(mask_ndti),np.mean(mask_rvi),np.mean(mask_gli),np.std(mask_ndvi),
                               np.mean(mask_bsi),np.mean(mask_nir),np.mean(mask_mcari),np.mean(mask_savi),
                               np.mean(mask_grndvi),np.mean(mask_ndsi),np.mean(mask_gb),np.mean(mask_psri),
                               np.mean(mask_npci),np.mean(mask_ccci),np.mean(mask_gcvi),np.mean(mask_red),
                               np.mean(mask_edge), np.mean(mask_green),np.mean(mask_blue),np.mean(mask_edge2),
                               np.mean(mask_edge3),np.mean(mask_8a),np.std(mask_nir),
                               np.mean(mask_clred_edge),np.mean(mask_slavi),np.mean(mask_datt1),np.mean(mask_arvi)])



      if ((date==month0)& (f==1)):
        train=pd.DataFrame(field_ndvi,columns=['tile_id','field_id',
                                              f'vapor_{int(date)}_{f}',f'ndvi_{int(date)}_{f}',f'lswi_{int(date)}_{f}',
                                              f'ndsvi_{int(date)}_{f}',f'ndti_{int(date)}_{f}',f'rvi_{int(date)}_{f}',f'gli_{int(date)}_{f}',
                                              f'ndvi_std_{int(date)}_{f}',f'bsi_{int(date)}_{f}',f'nir_{int(date)}_{f}',
                                              f'mcari_{int(date)}_{f}',f'savi_{int(date)}_{f}',f'grndvi_{int(date)}_{f}',f'ndsi_{int(date)}_{f}',
                                              f'gb_{int(date)}_{f}',f'psri_{int(date)}_{f}',f'npci_{int(date)}_{f}',
                                              f'ccci_{int(date)}_{f}',f'gcvi_{int(date)}_{f}',f'red_{int(date)}_{f}',f'edge_{int(date)}_{f}',
                                              f'green_{int(date)}_{f}',f'blue_{int(date)}_{f}',f'edge2_{int(date)}_{f}',
                                              f'edge3_{int(date)}_{f}',f'8a_{int(date)}_{f}',f'nir_std_{int(date)}_{f}',f'c1_{int(date)}_{f}',
                                              f'c2_{int(date)}_{f}',f'c3_{int(date)}_{f}',f'c4_{int(date)}_{f}'])
      else: 
        train[[f'vapor_{int(date)}_{f}',f'ndvi_{int(date)}_{f}',f'lswi_{int(date)}_{f}',
               f'ndsvi_{int(date)}_{f}',f'ndti_{int(date)}_{f}',f'rvi_{int(date)}_{f}',f'gli_{int(date)}_{f}',
            f'ndvi_std_{int(date)}_{f}',f'bsi_{int(date)}_{f}',f'nir_{int(date)}_{f}',
            f'mcari_{int(date)}_{f}',f'savi_{int(date)}_{f}',f'grndvi_{int(date)}_{f}',f'ndsi_{int(date)}_{f}',
            f'gb_{int(date)}_{f}',f'psri_{int(date)}_{f}',f'npci_{int(date)}_{f}',
            f'ccci_{int(date)}_{f}',f'gcvi_{int(date)}_{f}',f'red_{int(date)}_{f}',f'edge_{int(date)}_{f}',
            f'green_{int(date)}_{f}',f'blue_{int(date)}_{f}',f'edge2_{int(date)}_{f}',
            f'edge3_{int(date)}_{f}',f'8a_{int(date)}_{f}',f'nir_std_{int(date)}_{f}',f'c1_{int(date)}_{f}',
            f'c2_{int(date)}_{f}',f'c3_{int(date)}_{f}',f'c4_{int(date)}_{f}']]=month_i

  

  return train

###########################

In [ ]:
#executing this cell will take time, the output of this cell is cloudy(dataframe) which can be directly obtained
#by executing the next cell
train_fortnight_bf=veg_indices(cloud_free_train,info)

In [55]:
train1_ft=pd.read_csv('/content/drive/MyDrive/spot_the_crop/train1_ft_safe.csv')
train2_ft=pd.read_parquet('/content/drive/MyDrive/spot_the_crop/train2_ft_safe.parquet')
train3_ft=pd.read_parquet('/content/drive/MyDrive/spot_the_crop/train3_ft_safe.parquet')
train4_ft=pd.read_parquet('/content/drive/MyDrive/spot_the_crop/train4_ft_safe.parquet')

#concatenation
train_fortnight_bf=pd.concat([train1_ft, train2_ft,train3_ft,train4_ft], ignore_index=True)

###########################

**compactness feature**

In [84]:
import scipy.ndimage as ndi
import random
def check_circle(image):
 
  
  cy,cx=ndi.center_of_mass(image)
 
  a=np.array([cy,cx])
  arr=np.array([i for i in range(256)])
  max_0=np.argmax(image,axis=0)

  y=max_0[max_0>0]
  x=arr[max_0>0]
  if (len(y)<2):
    return 50
  distances=[]  
  for i in range(0,5):
    n = random.randint(0,len(y)-1)
    
    b=np.array([y[n],x[n]])
    distances.append(np.linalg.norm(a-b))

  return np.std(distances)
def circle_search(data,info,month0=4):
  field_ndvi=[]
  first=data[data.month==month0]
  
  
  for tile in first.tile_id.unique():

    tile_df=first[(first.tile_id==tile) ]
      
    src_nir=rasterio.open(tile_df[tile_df['asset']=='B08']['file_path'].values[0])
      
      


    tile_fields=rasterio.open(info[(info.tile_id==tile)&(info.asset=='field_ids')]['file_path'].values[0]).read(1)
    ids=np.unique(tile_fields.flatten())

    for field in ids:
      if (field==0):
        continue
      mask_binary=np.where(tile_fields==field,255,0).reshape(tile_fields.shape)    
          
      field_ndvi.append((field,check_circle(mask_binary),check_circle(mask_binary[::-1])))

        



    
  train=pd.DataFrame(field_ndvi,columns=['field_id',f'check_circle_top',f'check_circle_down'])
    

  

  return train

In [85]:
circles_check=circle_search(cloud_free_train1,info)

In [87]:
#add circles_check data
train_fortnight_bf=pd.merge(train_fortnight_bf,circles_check,on='field_id',how='left')

In [88]:
#add sentinel1 data
train_fortnight_bf=pd.merge(train_fortnight_bf,sentinel1_ft,on='field_id',how='left')

In [89]:
train_fortnight_bf.drop('tile_id',axis=1,inplace=True)

In [ ]:
#test_fortnight_bf=pd.read_parquet('/content/drive/MyDrive/spot_the_crop/test_fortnight_ahh.parquet')

In [91]:
#labels
train_fortnight_bf=pd.merge(train_fortnight_bf,label_df,on='field_id',how='left')

In [94]:
train_fortnight_bf.to_parquet('/content/drive/MyDrive/spot_the_crop/final_train_processing_XL.parquet')